In [1]:
from fancyimpute import BiScaler, KNN, NuclearNormMinimization, SoftImpute
from sklearn.ensemble import RandomForestRegressor
import pandas as pd
import numpy as np


Using TensorFlow backend.


In [2]:
train_df = pd.read_csv("./train_data_1112.csv")
test_df = pd.read_csv("./test_data_1112.csv")
#train_df = pd.DataFrame(KNN(k=6).fit_transform(data)) 
#train_df.columns = ['sex','age','label']  # fancyimpute填补缺失值时会自动删除列名


In [3]:
train_df.isnull().sum()

poi_id                    0
request_id                0
pos                       0
time                      0
action                    0
uuid                      0
request_cate_id           0
request_time              0
latitude_req              0
longitude_req             0
device_type            1664
gender               335217
age                 1640251
job                       0
cate_level1               0
cate_level2               0
cate_level3               0
area_id                   0
avg_price               241
poi_star               1477
longitude_poi             0
latitude_poi              0
distance                  0
peak                      0
poi_cnt_deal         162271
poi_avg_discount     162271
dtype: int64

In [12]:
train_df.head()

,poi_id,request_id,pos,time,action,uuid,request_cate_id,request_time,latitude_req,longitude_req,...,cate_level3,area_id,avg_price,poi_star,longitude_poi,latitude_poi,distance,peak,poi_cnt_deal,poi_avg_discount
0,1702902487627383470,4311086084099128885,13,2019-05-02,0,9.639066e+18,1.0,16,29.178693,105.751887,...,2036,15743,66.0,43.0,105.716693,29.167927,3.621,0,5.0,0.737855
1,14800569415264211896,4749834003306548820,1,2019-05-02,0,1.328961e+18,1.0,19,38.003857,105.462191,...,244,31806,49.0,37.0,105.440202,38.060255,6.560,1,4.0,0.791588
2,6279863632342018475,13389247690348720622,83,2019-05-02,0,1.069036e+19,1.0,17,22.049558,112.603200,...,12,6941,49.0,48.0,112.568717,22.080556,4.951,1,7.0,0.733437
3,11127900293242244072,3988416664035581676,43,2019-05-02,0,3.636211e+18,1.0,22,30.225835,120.408761,...,321,35343,100.0,39.0,119.348778,29.815176,111.802,0,3.0,0.571207
4,15471065569104080361,17230175878920950111,13,2019-05-02,0,4.344248e+18,1.0,18,43.411473,86.729880,...,235,34029,107.0,50.0,86.770657,43.404640,3.380,1,7.0,0.535113


In [4]:
train_df_user = train_df[['uuid','gender','age','job']]
train_df_poi  = train_df[['poi_id','cate_level1','cate_level2','cate_level3','area_id','avg_price','poi_star','longitude_poi','latitude_poi']]

In [7]:
train_df_user.head()

,uuid,gender,age,job
0,9.639066e+18,0.0,NaN,1.0
1,1.328961e+18,0.0,NaN,3.0
2,1.069036e+19,1.0,20.0,3.0
3,3.636211e+18,0.0,18.0,2.0
4,4.344248e+18,0.0,31.0,1.0


In [9]:
known_age = train_df_user[train_df_user.age.notnull()].as_matrix()

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [24]:
known_age[0,[2,3]]

array([20.,  3.])

In [ ]:
#先填最大的，把其他数量小的特征都补起来
train_df_user_fit_g = train_df_user.fillna(train_df.mean()['gender'])
def fill_ages(df):
    #把数值型特征都放到随机森林里面去
    known_age = df[df.age.notnull()].as_matrix()
    unknown_age = df[df.age.isnull()].as_matrix()
    y=known_age[:,2]#y是年龄，第一列数据
    x=known_age[:,[0,1,3]]#x是特征属性值，后面几列
    rfr=RandomForestRegressor(random_state=0,n_estimators=100,n_jobs=-1)
    #根据已有数据去拟合随机森林模型
    rfr.fit(x,y)
    #预测缺失值
    predictedAges = rfr.predict(unknown_age[:,1:])
    #填补缺失值
    df.loc[(df.age.isnull()),'age'] = predictedAges
    return df,rfr

train_df_user_fit_g=fill_ages(train_df_user_fit_g)

In [25]:
#在填年龄之前先把其他的用均值填掉
def fill_gender(df):
    #把数值型特征都放到随机森林里面去
    known_g = df[df.gender.notnull()].as_matrix()
    unknown_g = df[df.gender.isnull()].as_matrix()
    y=known_g[:,1]#y是年龄，第一列数据
    x=known_g[:,[0,2,3]]#x是特征属性值，后面几列
    rfr=RandomForestRegressor(random_state=0,n_estimators=100,n_jobs=-1)
    #根据已有数据去拟合随机森林模型
    rfr.fit(x,y)
    #预测缺失值
    predictedg = rfr.predict(unknown_g[:,[0,2,3]])
    #填补缺失值
    df.loc[(df.gender.isnull()),'gender'] = predictedg
    return df,rfr

fill_gender (train_df_user)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.


ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [ ]:
train_user_fit = pd.DataFrame(KNN(k=5).fit_transform(train_df_user)) 
#train_user_fit = KNN(k=3).complete(train_df_user)

In [ ]:
train_user_fit.head()

In [ ]:
train_poi_fit = pd.DataFrame(KNN(k=10).fit_transform(train_df_poi)) 